In [ ]:
# For package management
workdir_ = '/glade/work/juliob/'
import sys
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Get some regirdding stuff
from PyRegridding.Regridder import scripGen as SG

# Some other useful packages 
import importlib
#import esmpy as E

# reload dev codes
importlib.reload( SG )



In [ ]:

SST2 = xr.open_dataset('/glade/campaign/cgd/amp/juliob/NOAA_OI_SST/sst_ice_NOAA_QxQ_2003_2005.nc' )
lon2=SST2.lon.values
lat2=SST2.lat.values


dso=SG.latlon_to_scrip( longitudes=lon2, latitudes=lat2 , file_out='NOAA_OI_SST_scrip.nc', SkipAreaCheck=False )

#dso=SG.latlon_to_scrip( myGrid=True, dx=0.25, dy=0.25, file_out='NOAA_OI_SST_scrip_v2.nc', SkipAreaCheck=True )

In [ ]:
def scrip_to_esmf_mesh(scrip_file, output_mesh_file):
    """
    Convert a SCRIP file into an ESMF mesh file.
    
    Parameters:
    scrip_file : str
        Path to the SCRIP file (NetCDF) containing grid data.
    output_mesh_file : str
        Path to output the generated ESMF mesh file.
    """
    # Open the SCRIP file with xarray
    ds = xr.open_dataset(scrip_file)
    
    # Extract corner coordinates from the SCRIP file
    x_corner = ds['grid_corner_lon'].values  # shape (720, 1440, 4)
    y_corner = ds['grid_corner_lat'].values  # shape (720, 1440, 4)
    
    # Flatten the corner arrays into a 2D array (each corner becomes a unique pair of coordinates)
    x_flat = x_corner.flatten()
    y_flat = y_corner.flatten()

    # Combine longitude and latitude into one array of coordinates
    nodeCoords = np.column_stack((x_flat, y_flat))
    
    # Extract unique node coordinates and map original corner points to these nodes
    unique_nodeCoords, node_indices = np.unique(nodeCoords, axis=0, return_inverse=True)
    
    # Count the number of unique nodes
    nodeCount = unique_nodeCoords.shape[0]
    
    # Initialize the ESMF mesh object
    mesh = E.Mesh(parametric_dim=2, spatial_dim=2)
    
    # Add the nodes (unique corner coordinates)
    mesh.add_nodes(nodeCount, unique_nodeCoords)
    
    # Reshape the node indices to the original grid structure, mapping corners to cells
    node_indices = node_indices.reshape(x_corner.shape)
    
    # Create the element connectivity array (each row has the 4 node indices for one cell)
    elementConn = np.empty((x_corner.shape[0] * x_corner.shape[1], 4), dtype=int)

    for i in range(x_corner.shape[0]):
        for j in range(x_corner.shape[1]):
            elementConn[i * x_corner.shape[1] + j] = node_indices[i, j, :]
    
    # Add elements (cells) to the mesh using the connectivity information
    mesh.add_elements(x_corner.shape[0] * x_corner.shape[1], elementConn)
    
    # Write the mesh to a file in ESMF mesh format
    mesh.write(filename=output_mesh_file, filetype=E.FileFormat.ESMFMESH)
    
    print(f"ESMF mesh file written to {output_mesh_file}")


In [ ]:
scrip_file = 'NOAA_OI_SST_scrip.nc'
output_mesh_file = 'output_esmf_mesh.nc'
scrip_to_esmf_mesh(scrip_file, output_mesh_file)

In [ ]:
def scrip_to_esmf_mesh_v2(scrip_file, output_mesh_file):
    """
    Convert a SCRIP file into an ESMF mesh file.
    
    Parameters:
    scrip_file : str
        Path to the SCRIP file (NetCDF) containing grid data.
    output_mesh_file : str
        Path to output the generated ESMF mesh file.
    """
    # Open the SCRIP file with xarray
    ds = xr.open_dataset(scrip_file)
    
    # Extract corner coordinates from the SCRIP file
    x_corner = ds['grid_corner_lon'].values  # shape (720, 1440, 4)
    y_corner = ds['grid_corner_lat'].values  # shape (720, 1440, 4)
    
    # Flatten the corner arrays into a 2D array (each corner becomes a unique pair of coordinates)
    x_flat = x_corner.flatten()
    y_flat = y_corner.flatten()

    # Combine longitude and latitude into one array of coordinates
    nodeCoords = np.column_stack((x_flat, y_flat))
    
    # Extract unique node coordinates and map original corner points to these nodes
    unique_nodeCoords, node_indices = np.unique(nodeCoords, axis=0, return_inverse=True)

    print( node_indices.shape )
    
    # Count the number of unique nodes
    nodeCount = unique_nodeCoords.shape[0]
    
    # All nodes are owned by processor 0 (single processor case)
    nodeOwners = np.zeros(nodeCount, dtype=int)

    # Create a mask for the nodes (0 means no mask, all nodes are included)
    nodeMask = np.zeros(nodeCount, dtype=int)

    # Initialize the ESMF mesh object
    mesh = E.Mesh(parametric_dim=2, spatial_dim=2)
    
    # Add the nodes (unique corner coordinates)
    #mesh.add_nodes(nodeCount, unique_nodeCoords, nodeOwners)
    mesh.add_nodes(nodeCount, unique_nodeCoords, nodeOwners, nodeMask)

    # Reshape the node indices to the original grid structure, mapping corners to cells
    #node_indices = node_indices.reshape(x_corner.shape)
    # Reshape the node indices to match the original corner array
    print( x_corner.shape )
    node_indices = node_indices.reshape(x_corner.shape[0], x_corner.shape[1], 4)

    # Create the element connectivity array (each row has the 4 node indices for one cell)
    elementConn = np.empty((x_corner.shape[0] * x_corner.shape[1], 4), dtype=int)

    for i in range(x_corner.shape[0]):
        for j in range(x_corner.shape[1]):
            elementConn[i * x_corner.shape[1] + j] = node_indices[i, j, :]
    
    # Add elements (cells) to the mesh using the connectivity information
    mesh.add_elements(x_corner.shape[0] * x_corner.shape[1], elementConn)
    
    # Write the mesh to a file in ESMF mesh format
    mesh.write(filename=output_mesh_file, filetype=E.FileFormat.ESMFMESH)
    
    print(f"ESMF mesh file written to {output_mesh_file}")

In [ ]:
print(720*1440)

In [ ]:
# Example usage
scrip_file = 'NOAA_OI_SST_scrip.nc'
output_mesh_file = 'output_esmf_mesh.nc'
scrip_to_esmf_mesh_v2(scrip_file, output_mesh_file)

In [ ]:
def scrip_to_esmf_mesh_v3(scrip_file, output_mesh_file):
    """
    Convert a SCRIP file into an ESMF mesh file.
    
    Parameters:
    scrip_file : str
        Path to the SCRIP file (NetCDF) containing grid data.
    output_mesh_file : str
        Path to output the generated ESMF mesh file.
    """
    # Open the SCRIP file with xarray
    ds = xr.open_dataset(scrip_file)
    
    # Extract corner coordinates from the SCRIP file
    x_corner = ds['grid_corner_lon'].values  # shape (1036800, 4)
    y_corner = ds['grid_corner_lat'].values  # shape (1036800, 4)
    
    # Flatten the corner arrays into a 2D array (each corner becomes a unique pair of coordinates)
    x_flat = x_corner.flatten()
    y_flat = y_corner.flatten()

    # Combine longitude and latitude into one array of coordinates
    nodeCoords = np.column_stack((x_flat, y_flat))
    
    # Extract unique node coordinates and map original corner points to these nodes
    unique_nodeCoords, node_indices = np.unique(nodeCoords, axis=0, return_inverse=True)
    
    # Count the number of unique nodes
    nodeCount = unique_nodeCoords.shape[0]
    
    # All nodes are owned by processor 0 (single processor case)
    nodeOwners = np.zeros(nodeCount, dtype=int)
    
    # Create a mask for the nodes (0 means no mask, all nodes are included)
    nodeMask = np.zeros(nodeCount, dtype=int)
    
    # Initialize the ESMF mesh object
    mesh = E.Mesh(parametric_dim=2, spatial_dim=2)
    
    # Add the nodes (unique corner coordinates)
    mesh.add_nodes(nodeCount, unique_nodeCoords, nodeOwners, nodeMask)
    



    # `node_indices` is already flattened, and we can directly map it to the corners of each cell.
    # Each grid cell has 4 corner indices, and we already know the shape is (1036800, 4)
    elementConn = node_indices.reshape(x_corner.shape)
    
    # All elements are quadrilateral, so we create an array for element types
    element_types = np.full(x_corner.shape[0], E.MeshElemType.QUAD)
    
    # Create an element mask (0 means no masking, all elements are included)
    element_mask = np.zeros(x_corner.shape[0], dtype=int)
    
    # Add elements (cells) to the mesh using the connectivity information, element types, and mask
    mesh.add_elements(x_corner.shape[0], element_types, elementConn, element_mask)

    
    # Write the mesh to a file in ESMF mesh format
    mesh.write(filename=output_mesh_file, filetype=E.FileFormat.ESMFMESH)
    
    print(f"ESMF mesh file written to {output_mesh_file}")

In [ ]:
# Example usage
scrip_file = 'NOAA_OI_SST_scrip.nc'
output_mesh_file = 'output_esmf_mesh.nc'
scrip_to_esmf_mesh_v3(scrip_file, output_mesh_file)